In [158]:
import geopandas as gp
import geemap
# import geemap.foliumap as geemap
import ee
ee.Initialize()

In [159]:
# Algorithm constants
ndvi_v = 0.6 # vegetation
ndvi_s = 0.2 # soil

# emissivity values
epsilon_v = 0.985
epsilon_s = 0.97
epsilon_w = 0.99

# Constants
cs_l8 = [0.04019, 0.02916, 1.01523,
         -0.38333, -1.50294, 0.20324,
         0.00918, 1.36072, -0.27514]
cs_l7 = [0.06518, 0.00683, 1.02717,
         -0.53003, -1.25866, 0.10490,
         -0.01965, 1.36947, -0.24310]
cs_l5 = [0.07518, -0.00492, 1.03189,
         -0.59600, -1.22554, 0.08104,
         -0.02767, 1.43740, -0.25844]

In [160]:
# Time
year_start = 2020
year_end = 2020
month_start = 5
month_end = 8

# Space
aois = gp.read_file("../../data/feature_layers/aois_test.geojson")

bbox = list(aois.total_bounds)
select_roi = ee.Geometry.Rectangle(bbox)
max_cloud_cover = 60
epsg = 'EPSG:32655'
pixel_resolution = 30
roi_filename = 'KYT_TEST'

In [198]:
# Rename bands
def fun_bands(img):
    bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
    thermal_band = ['ST_B10']
    new_bands = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2']
    new_thermal_bands = ['TIR']
    vnirswir = img.select(bands)\
        .multiply(0.0000275).add(-0.2)\
            .rename(new_bands)
    tir = img.select(thermal_band)\
        .multiply(0.00341802).add(149.0).subtract(273.15)\
            .rename(new_thermal_bands)
    return vnirswir.addBands(tir).copyProperties(img, ['system:time_start'])

In [162]:
# Mask OLI-TIRS images 
def fun_mask_ls_sr(img):
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()
    snowBitMask = ee.Number(2).pow(4).int()
    qa = img.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
        qa.bitwiseAnd(cloudsBitMask).eq(0)).And(
            qa.bitwiseAnd(snowBitMask).eq(0)
            )
    return img.updateMask(mask)

In [163]:
# Radiometric Calibration
def fun_radcal(img):
    radiance = ee.Algorithms.Landsat.calibratedRadiance(img).rename('RADIANCE')
    return img.addBands(radiance)


# L to ee.Image
def fun_l_addband(img):
    l = ee.Image(img.get('L')).select('RADIANCE').rename('L')
    return img.addBands(l)


# Create maxDifference-filter to match TOA and SR products
maxDiffFilter = ee.Filter.maxDifference(
    difference=2 * 24 * 60 * 60 * 1000,
    leftField= 'system:time_start',
    rightField= 'system:time_start'
)

# Define join: Water vapor
join_wv = ee.Join.saveBest(
    matchKey = 'WV',
    measureKey = 'timeDiff'
)

# Define join: Radiance
join_l = ee.Join.saveBest(
    matchKey = 'L',
    measureKey = 'timeDiff'
)

In [164]:
# calculate NDVI
def fun_ndvi(img):
    ndvi = img.normalizedDifference(['NIR', 'R']).rename('NDVI')
    return img.addBands(ndvi)

# Fraction Vegetation Cover (FVC)
def fun_fvc(img):
    fvc = img.expression(
        '((NDVI-NDVI_s)/(NDVI_v-NDVI_s))**2',
        {
            'NDVI': img.select('NDVI'),
            'NDVI_s': ndvi_s,
            'NDVI_v': ndvi_v
        }
    ).rename('FVC')
    return img.addBands(fvc)

In [165]:
# Scale Emissivity (Epsilon) between NDVI_s and NDVI_v
def fun_epsilon_scale(img):
    epsilon_scale = img.expression(
        'epsilon_s+(epsilon_v-epsilon_s)*FVC',
        {
            'FVC': img.select('FVC'),
            'epsilon_s': epsilon_s,
            'epsilon_v': epsilon_v
        }
    ).rename('EPSILON_SCALE')
    return img.addBands(epsilon_scale)

In [166]:
# Get emissivity from NDVI thresholding
def fun_epsilon(img):
    pseudo = img.select(['NDVI']).set('system:time_start', img.get('system:time_start'))
    epsilon = pseudo.where(img.expression('NDVI > NDVI_v',
                                          {'NDVI': img.select('NDVI'),
                                           'NDVI_v': ndvi_v}), epsilon_v)
    epsilon = epsilon.where(img.expression('NDVI < NDVI_s && NDVI >= 0',
                                           {'NDVI': img.select('NDVI'),
                                            'NDVI_s': ndvi_s}), epsilon_s)
    epsilon = epsilon.where(img.expression('NDVI < 0',
                                           {'NDVI': img.select('NDVI')}), epsilon_w)
    epsilon = epsilon.where(img.expression('NDVI <= NDVI_v && NDVI >= NDVI_s',
                                           {'NDVI': img.select('NDVI'),
                                            'NDVI_v': ndvi_v,
                                            'NDVI_s': ndvi_s}), img.select('EPSILON_SCALE')).rename('EPSILON')
    return img.addBands(epsilon)


In [167]:
# Function to scale WV content product
def fun_wv_scale(img):
    wv_scaled = ee.Image(img.get('WV')).multiply(0.1).rename('WV_SCALED')
    wv_scaled = wv_scaled.resample('bilinear')
    return img.addBands(wv_scaled)


In [168]:
# Atmospheric Functions
def fun_af1(cs):
    def wrap(img):
        af1 = img.expression(
            '('+str(cs[0])+'*(WV**2))+('+str(cs[1])+'*WV)+('+str(cs[2])+')',
            {
                'WV': img.select('WV_SCALED')
            }
        ).rename('AF1')
        return img.addBands(af1)
    return wrap


def fun_af2(cs):
    def wrap(img):
        af2 = img.expression(
            '('+str(cs[3])+'*(WV**2))+('+str(cs[4])+'*WV)+('+str(cs[5])+')',
            {
                'WV': img.select('WV_SCALED')
            }
        ).rename('AF2')
        return img.addBands(af2)
    return wrap


def fun_af3(cs):
    def wrap(img):
        af3 = img.expression(
            '('+str(cs[6])+'*(WV**2))+('+str(cs[7])+'*WV)+('+str(cs[8])+')',
            {
                'WV': img.select('WV_SCALED')
            }
        ).rename('AF3')
        return img.addBands(af3)
    return wrap


In [169]:
# Gamma Functions
def fun_gamma_l8(img):
    gamma = img.expression('(BT**2)/(1324*L)',
                           {'BT': img.select('TIR'),
                            'L': img.select('L')
                            }).rename('GAMMA')
    return img.addBands(gamma)

In [170]:
# Delta Functions
def fun_delta_l8(img):
    delta = img.expression('BT-((BT**2)/1324)',
                           {'BT': img.select('TIR')
                            }).rename('DELTA')
    return img.addBands(delta)

In [171]:
# Land Surface Temperature
def fun_lst(img):
    lst = img.expression(
        '(GAMMA*(((1/EPSILON)*(AF1*L+AF2))+AF3)+DELTA)-273.15',
        {
            'GAMMA': img.select('GAMMA'),
            'DELTA': img.select('DELTA'),
            'EPSILON': img.select('EPSILON'),
            'AF1': img.select('AF1'),
            'AF2': img.select('AF2'),
            'AF3': img.select('AF3'),
            'L': img.select('L')
        }
    ).rename('LST')
    return img.addBands(lst)


In [172]:
def fun_date(img):
    return ee.Date(ee.Image(img).date().format("YYYY-MM-dd"))


def fun_getdates(imgCol):
    return ee.List(imgCol.toList(imgCol.size()).map(fun_date))


def fun_mosaic(date, newList):
    # cast list & date
    newList = ee.List(newList)
    date = ee.Date(date)

    # filter img-collection
    filtered = ee.ImageCollection(subCol.filterDate(date, date.advance(1, 'day')))

    # check duplicate
    img_previous = ee.Image(newList.get(-1))
    img_previous_datestring = img_previous.date().format("YYYY-MM-dd")
    img_previous_millis = ee.Number(ee.Date(img_previous_datestring).millis())

    img_new_datestring = filtered.select(parameter).first().date().format("YYYY-MM-dd")
    img_new_date = ee.Date(img_new_datestring).millis()
    img_new_millis = ee.Number(ee.Date(img_new_datestring).millis())

    date_diff = img_previous_millis.subtract(img_new_millis)

    # mosaic
    img_mosaic = ee.Algorithms.If(
        date_diff.neq(0),
        filtered.select(parameter).mosaic().set('system:time_start', img_new_date),
        ee.ImageCollection(subCol.filterDate(pseudodate, pseudodate.advance(1, 'day')))
    )

    tester = ee.Algorithms.If(date_diff.neq(0), ee.Number(1), ee.Number(0))

    return ee.Algorithms.If(tester, newList.add(img_mosaic), newList)


def fun_timeband(img):
    time = ee.Image(img.metadata('system:time_start', 'TIME').divide(86400000))
    timeband = time.updateMask(img.select(parameter).mask())
    return img.addBands(timeband)

# Apply on EE Server

In [173]:
ndvi_v = ee.Number(ndvi_v)
ndvi_s = ee.Number(ndvi_s)

epsilon_v = ee.Number(epsilon_v)
epsilon_s = ee.Number(epsilon_s)
epsilon_w = ee.Number(epsilon_w)

In [174]:
# Landsat 8 OLI-TIRS
imgCol_L8_TOA = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
    .filterBounds(select_roi)\
    .filter(ee.Filter.calendarRange(year_start,year_end,'year'))\
    .filter(ee.Filter.calendarRange(month_start,month_end,'month'))\
    .filter(ee.Filter.lt('CLOUD_COVER_LAND', max_cloud_cover))\
    .select(['B10'])

imgCol_L8_SR = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    .filterBounds(select_roi)\
    .filter(ee.Filter.calendarRange(year_start,year_end,'year'))\
    .filter(ee.Filter.calendarRange(month_start,month_end,'month'))\
    .filter(ee.Filter.lt('CLOUD_COVER_LAND', max_cloud_cover))\
    .map(fun_mask_ls_sr)

imgCol_L8_SR = imgCol_L8_SR.map(fun_bands)

In [175]:
# NCEP/NCAR Water Vapor Product
imgCol_WV = ee.ImageCollection('NCEP_RE/surface_wv')\
    .filterBounds(select_roi)\
    .filter(ee.Filter.calendarRange(year_start,year_end,'year'))\
    .filter(ee.Filter.calendarRange(month_start,month_end,'month'))


Get LST ImageCollection

In [199]:
imgCol_L8_SR = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
    .filterBounds(select_roi)\
    .filter(ee.Filter.calendarRange(year_start,year_end,'year'))\
    .filter(ee.Filter.calendarRange(month_start,month_end,'month'))\
    .filter(ee.Filter.lt('CLOUD_COVER_LAND', max_cloud_cover))\
    .map(fun_mask_ls_sr)

# Rename bands
imgCol_L8_SR = imgCol_L8_SR.map(fun_bands)

imgCol_L8_SR = imgCol_L8_SR.map(fun_ndvi)
imgCol_L8_SR = imgCol_L8_SR.map(fun_fvc)

In [195]:
import json
file_path = "../../data/feature_layers/roi.geojson"

with open(file_path) as f:
    json_data = json.load(f)

bbox = geemap.geojson_to_ee(json_data)

from ipyleaflet import GeoJSON
json_layer = GeoJSON(
    data=json_data,
    name="bbox",
    hover_style={"fillColor": "red", "fillOpacity": 0},
)


In [203]:
m = geemap.Map()
m.add_basemap("SATELLITE")
vis_params = {
    'bands': ['TIR'],
    'min': -5,
    'max': 40,
    'palette': "inferno",
}
m.add_layer(imgCol_L8_SR, vis_params, 'LST')
m.add_layer(json_layer)
m.setCenter(145.3421989895628, 70.8852923280616, zoom = 8)
m

Map(center=[70.8852923280616, 145.3421989895628], controls=(WidgetControl(options=['position', 'transparent_bg…

In [ ]:
 # Mosaic imgCollection
pseudodate = ee.Date('1960-01-01')
subCol = ee.ImageCollection(imgCol_merge.select(parameter))
dates = fun_getdates(subCol)
ini_date = ee.Date(dates.get(0))
ini_merge = subCol.filterDate(ini_date, ini_date.advance(1, 'day'))
ini_merge = ini_merge.select(parameter).mosaic().set('system:time_start', ini_date.millis())
ini = ee.List([ini_merge])
imgCol_mosaic = ee.ImageCollection(ee.List(dates.iterate(fun_mosaic, ini)))
imgCol_mosaic = imgCol_mosaic.map(fun_timeband)